In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [2]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [3]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [4]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [5]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [6]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [7]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g2024885-d5895975-Reviews-Shilla_Stay_Dongtan-Hwaseong_Gyeonggi_do.html"

data_sample = hotel_review_crawling(url)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
26 번째 페이지
27 번째 페이지
28 번째 페이지
29 번째 페이지
30 번째 페이지
31 번째 페이지
32 번째 페이지
33 번째 페이지
34 번째 페이지
35 번째 페이지
36 번째 페이지
37 번째 페이지
38 번째 페이지
39 번째 페이지
40 번째 페이지
41 번째 페이지
42 번째 페이지
43 번째 페이지
44 번째 페이지
45 번째 페이지
46 번째 페이지
47 번째 페이지
48 번째 페이지
49 번째 페이지
50 번째 페이지
51 번째 페이지


,nickname,score,review,stay_date
0,송정훈,50,주변 상권이 먹자골목이라 주변에 먹을곳이 많아요\n깨끗하고 깔끔하지만 방음이 조금 ...,2021-03-01
1,yeona,50,서비스 친절하고 시설 및 침구류 깨끗하고 위치적으로도 너무 좋음 무엇보다 아베다 어...,2021-04-01
2,Jeonghwa,50,"창밖으로 공원이 보이는 뷰로 시야가 틔여서 답답하지 않았구요, 마침 오늘 비오는 날...",2021-04-01
3,moody,50,룸컨디션 깔끔하고 좋습니다. 시티뷰라서 주변 뷰가 좋은편은 아니지만 창문 옆에 길게...,2021-03-01
4,Taegyu won,50,둘이 쓰기에 방 크기가 넉넉하고 너무나도 쾌적하고 좋습니다! 주차공간도 넉넉하여 주...,2021-02-01
...,...,...,...,...
250,sung288,50,근처에 살면서 1층 카페만 이용해보다가 지방에서 친척분들 올라오시면서 이용해보았는데...,2018-02-01
251,현진 김,40,시설도 깨끗하고 조식도 맛있었어요\n직원도 친절하고 좋았어요\n다만 비즈니스 호텔이...,2017-08-01
252,m0nica1216,50,주말에 혼자 쉬러 방문했습니다. 객실크기는 혼자나 둘이 쓰기 적당하고 샤워가운이 부...,2018-06-01
253,douner0707,40,늦은 시간 입실이었는데\n비교적 빨리 입실해서\n잘 쉬었습니다.\n조식도 무난했고요...,2018-05-01


In [8]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample.to_csv('./data/hotel_review_shillastay_dongtan_origin.csv', encoding="utf-8-sig")

In [10]:
data = preprocessing_data(data_sample)

In [11]:
data

,nickname,score,review,stay_date
0,Jeonghwa,50,"창밖으로 공원이 보이는 뷰로 시야가 틔여서 답답하지 않았구요, 마침 오늘 비오는 날...",2021-04-01
1,yeona,50,서비스 친절하고 시설 및 침구류 깨끗하고 위치적으로도 너무 좋음 무엇보다 아베다 어...,2021-04-01
2,송정훈,50,주변 상권이 먹자골목이라 주변에 먹을곳이 많아요 깨끗하고 깔끔하지만 방음이 조금 아...,2021-03-01
3,moody,50,룸컨디션 깔끔하고 좋습니다. 시티뷰라서 주변 뷰가 좋은편은 아니지만 창문 옆에 길게...,2021-03-01
4,Taegyu won,50,둘이 쓰기에 방 크기가 넉넉하고 너무나도 쾌적하고 좋습니다! 주차공간도 넉넉하여 주...,2021-02-01
...,...,...,...,...
246,Navigator819084,50,컨디션 너무 깔끔하고 다들 친절하셔서 너무 좋았어요ㅋㅋ 친구끼리 방문했는데 좋았습니...,2018-09-01
247,soyun9121,50,신라스테이호텔을 자주 이용하는데 동탄점은 처음 이였어요 역시나 서비스도 좋았고 직원...,2018-08-01
248,Jen C,40,생각보다 방도 작지 않았고 뷰도 좋았고(공원뷰?) 깔끔했어요. 조식은 이용하지 않아...,2018-08-01
249,지연 이,30,장점 저렴하고 깔끔함 방 크기가 비즈니스 호텔치고는 넓음 뷰 좋음 단점 협소한 주...,2018-08-01


In [12]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-12-7183346d9643>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [13]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-13-b71e843350f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [14]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_dongtan_final.csv', encoding="utf-8-sig")

<ipython-input-14-d7ffddcb6545>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [15]:
test_data

,nickname,score,review,stay_date
0,Jeonghwa,50,창밖으로 공원이 보이는 뷰로 시야가 틔여서 답답하지 않았구요 마침 오늘 비오는 날이...,2021-04-01
1,yeona,50,서비스 친절하고 시설 및 침구류 깨끗하고 위치적으로도 너무 좋음 무엇보다 아베다 어...,2021-04-01
2,송정훈,50,주변 상권이 먹자골목이라 주변에 먹을곳이 많아요 깨끗하고 깔끔하지만 방음이 조금 아...,2021-03-01
3,moody,50,룸컨디션 깔끔하고 좋습니다 시티뷰라서 주변 뷰가 좋은편은 아니지만 창문 옆에 길게 ...,2021-03-01
4,Taegyu won,50,둘이 쓰기에 방 크기가 넉넉하고 너무나도 쾌적하고 좋습니다 주차공간도 넉넉하여 주차...,2021-02-01
...,...,...,...,...
246,Navigator819084,50,컨디션 너무 깔끔하고 다들 친절하셔서 너무 좋았어요 친구끼리 방문했는데 좋았습니다 ...,2018-09-01
247,soyun9121,50,신라스테이호텔을 자주 이용하는데 동탄점은 처음 이였어요 역시나 서비스도 좋았고 직원...,2018-08-01
248,Jen C,40,생각보다 방도 작지 않았고 뷰도 좋았고공원뷰 깔끔했어요 조식은 이용하지 않아서 모르...,2018-08-01
249,지연 이,30,장점 저렴하고 깔끔함 방 크기가 비즈니스 호텔치고는 넓음 뷰 좋음 단점 협소한 주...,2018-08-01
